<a href="https://colab.research.google.com/github/alirouzbayani1/pericardiocentesis-cardiac-wall-detection/blob/main/wall_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/alirouzbayani1/pericardiocentesis-cardiac-wall-detection.git

In [ ]:
%cd pericardiocentesis-cardiac-wall-detection

In [ ]:
!wget http://models.tensorpack.com/HED/HED_reproduced.npz

In [ ]:
import cv2
import re
import os
from google.colab.patches import cv2_imshow
import numpy as np

In [ ]:
videos_path = './sample_videos/'
frames_path = './sample_frames/'
model_path = './HED_reproduced.npz'

In [ ]:
def video_to_frames(video_path, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0
    while success:
        cv2.imwrite(output_path + "/frame%d.jpg" % count, image)     # save frame as JPEG file      
        success, image = vidcap.read()
        count += 1
def videos_to_frames(videos_path, output_path):
    for filename in os.listdir(videos_path):
        output_dir = filename.split('.')[0]
        output_dir_path = os.path.join(output_path, output_dir)
        input = os.path.join(videos_path, filename)
        video_to_frames(input, output_dir_path)

videos_to_frames(videos_path, frames_path)

In [ ]:
!zip -r sample_frames.zip sample_frames

In [ ]:
!pip install tensorpack
from hed import *

In [ ]:
def hed_func(model_path, input_img=None, input_path=None, output_path=None):
    pred_config = PredictConfig(
        model=Model(),
        session_init=SmartInit(model_path),
        input_names=['image'],
        output_names=['output' + str(k) for k in range(1, 7)])
    predictor = OfflinePredictor(pred_config)
    if input_img is not None:
      im = input_img
    else:
      im = cv2.imread(input_path)
    assert im is not None
    im = cv2.resize(
        im, (im.shape[1] // 16 * 16, im.shape[0] // 16 * 16)
    )[None, :, :, :].astype('float32')
    outputs = predictor(im)
    pred = outputs[5][0] * 255
    if output_path:
        cv2.imwrite(output_path, pred)
    else:
        return pred

In [ ]:
image_path='/content/pericardiocentesis-cardiac-wall-detection/sample_frames/d/frame2.jpg'

In [ ]:
def preprocess(image_path, clahe):

  orig_img = cv2.imread(image_path)
  orig_img=orig_img[20:512-20,20:512-20]
  # Blur
  # blured = cv2.GaussianBlur(orig_img, (31, 31), 0)
  
  # CLAHE
  gray_orig = cv2.cvtColor(orig_img, cv2.COLOR_BGR2GRAY)
  framet = clahe.apply(gray_orig)
  rgb_clahe = cv2.cvtColor(framet, cv2.COLOR_GRAY2RGB)
  cv2_imshow(rgb_clahe)

  # HED
  # hed_img = hed_func(model_path, input_img=rgb_clahe)

  # Canny
  canny_img = cv2.Canny(orig_img, 30, 60)

  return canny_img
  
def inpaint(image_path, hed_img, clahe):

  orig_img = cv2.imread(image_path)
  orig_img=orig_img[20:512-20,20:512-20]

  cv2_imshow(hed_img)

  # Binary Threshold
  _, hed_binary = cv2.threshold(hed_img, 25, 255, cv2.THRESH_BINARY)

  # Dilate
  dilated = cv2.dilate(hed_binary, np.ones([9,9]), 1)
  cv2_imshow(dilated)


  # Inpainting
  inpaint = cv2.inpaint(orig_img, dilated, 2, cv2.INPAINT_NS)
  cv2_imshow(inpaint)

  # Gray
  gray_inpaint = cv2.cvtColor(inpaint, cv2.COLOR_BGR2GRAY)

  # CLAHE
  # framet = clahe.apply(gray_inpaint)

  return gray_inpaint




In [ ]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(10,10))
ans=preprocess(image_path, clahe);
cv2_imshow(ans)
in_ans=inpaint(image_path, ans, clahe);
cv2_imshow(in_ans)


In [ ]:
def frame_diffs(new_frame_path, clahe, prev_frame=None):
  preprocessed_img = preprocess(new_frame_path, clahe)
  inpainted = inpaint(new_frame_path, preprocessed_img, clahe)
  if prev_frame is not None:
    diff = cv2.subtract(prev_frame, inpainted)
    clahe_diff = clahe.apply(diff)
    # cv2_imshow(clahe_diff)
    return diff
  else:
    return inpainted

def wrapper(frames_dir_path):
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
  outputs = []
  for i, filename in enumerate(sorted(os.listdir(frames_dir_path), key=lambda f: int(re.sub('\D', '', f)))):
    image_path = os.path.join(frames_dir_path, filename)
    if i == 0:
      tmp_frame = frame_diffs(image_path, clahe)
    else:
      tmp_frame = frame_diffs(image_path, clahe=clahe, prev_frame=tmp_frame)
    outputs.append(tmp_frame)

  return outputs

In [ ]:
for dir in os.listdir('./sample_frames/'):
  dir_path = os.path.join('./sample_frames/', dir)
  for i, filename in enumerate(sorted(os.listdir(dir_path), key=lambda f: int(re.sub('\D', '', f)))):
    image_path = os.path.join(dir_path, filename)
    img = cv2.imread(image_path)
    hed_result = hed_func(model_path, input_img=img)
    if not os.path.exists(f"./hed_outputs_{dir}"):
      os.makedirs(f"./hed_outputs_{dir}")
    cv2.imwrite(f'./hed_outputs_{dir}/frame{i}.jpg', hed_result)

In [ ]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
for dir in os.listdir('./sample_frames/'):
  dir_path = os.path.join('./sample_frames/', dir)
  for i, filename in enumerate(sorted(os.listdir(dir_path), key=lambda f: int(re.sub('\D', '', f)))):
    image_path = os.path.join(dir_path, filename)
    preprocessed = preprocess(image_path, clahe)
    inpainted = inpaint(image_path, preprocessed, clahe)
    if not os.path.exists(f"./inpainted_outputs_{dir}"):
      os.makedirs(f"./inpainted_outputs_{dir}")
    cv2.imwrite(f'./inpainted_outputs_{dir}/frame{i}.jpg', inpainted)

In [ ]:
dir_path = './sample_frames/d'
for i, filename in enumerate(sorted(os.listdir(dir_path), key=lambda f: int(re.sub('\D', '', f)))):
  image_path = os.path.join(dir_path, filename)
  preprocessed = preprocess(image_path, clahe)
  inpainted = inpaint(image_path, preprocessed, clahe)
  if not os.path.exists(f"./inpainted_outputs_d"):
    os.makedirs(f"./inpainted_outputs_d")
  cv2.imwrite(f'./inpainted_outputs_d/frame{i}.jpg', inpainted)

In [ ]:
!zip -r hed_outputs_*{.zip,}

In [ ]:
outputs = main_loop('./sample_frames/b/')
for i, output in enumerate(outputs):
  cv2.imwrite("./outputs/frame%d" % i, output)

In [ ]:
out_image = hed_func('./sample_frames/a/frame0.jpg', model_path)
cv2_imshow(out_image)

In [ ]:
!python ./hed.py --load ./HED_reproduced.npz --run ./sample_frames/a/frame0.jpg